Task 1: Third-Order Letter Approximation Model

In [66]:
# Imports.
# Selecting random items from lists.
import random
# Efficient data structures.
import collections
import json

Step 1: A method that will read in the text from one of the books

In [67]:
#method to read in a book
def readBook(book):
    with open('books/'+book, 'r', encoding='utf-8') as file:#open the file
        text = file.read()#store the read in file in a variable
    return text#return the text

In [68]:
#Clean the text by putting it to lower case and only keeping ascii chars
def cleanBook(text):
    #change everything to lower case
    english = text.upper();
    # The characters to keep.
    keep = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ .'
    # Remove unwanted characters.
    cleaned = ''.join(c if c in keep else ' ' for c in english)
    cleaned = ' '.join(cleaned.split())
    #get rif of new lines note to self
    return cleaned


In [69]:
#method to remove the preamble and poatamble 
def getRidOfPreambleAndPostamble(book):
    #sentences that are at the start and end of the actual content
    startOfBook = "*** START OF THE PROJECT GUTENBERG EBOOK"
    endOfBook = "*** END OF THE PROJECT GUTENBERG EBOOK"
    #if the starter sentence is in the text
    if startOfBook in book:
        #split the text at the start line and keep the content after it
        book = book.split(startOfBook, 1)[1]
    #if the end sentence is in the text
    if endOfBook in book:
        #split the text at the end line and keep the content before it
        book = book.split(endOfBook, 1)[0]
    #return the book without the preamble and the postamble
    return book 

In [70]:
text = readBook('voyaging.txt')
strippedText = getRidOfPreambleAndPostamble(text)
cleanedText = cleanBook(strippedText)
print(cleanedText)

DAVID GOES VOYAGING DAVID GOES VOYAGING BY DAVID BINNEY PUTNAM WITH ILLUSTRATIONS FROM PHOTOGRAPHS AND DECORATIONS BY ISABEL COOPER DON DICKERMAN AND DWIGHT FRANKLIN G. P. PUTNAM S SONS NEW YORK AND LONDON THE KNICKERBOCKER PRESS COPYRIGHT BY G. P. PUTNAM S SONS FIRST PRINTING SEPTEMBER SECOND PRINTING OCTOBER THIRD PRINTING NOVEMBER FOURTH PRINTING NOVEMBER FIFTH PRINTING NOVEMBER MADE IN THE UNITED STATES OF AMERICA TO GRANDMA AND GRANDPA BUB A SOLEMN FOREWORD MR. GEORGE PUTNAM HAS ASKED ME TO WRITE A SOLEMN FOREWORD TO DAVID S BOOK. AN IDEAL FOREWORD CANNOT BE TOO BRIEF SHOULD NEVER BE IN THE NATURE OF AN APOLOGY OR A PANEGYRIC AND IN FACT ANY DIRECT REFERENCE TO THE SUBJECT OF THE VOLUME IN WHICH IT IS PRINTED IS IN THE NATURE OF REDUNDANCY. ITS ONLY USE AS FAR AS I CAN SEE IS A CHANCE TO EXPLOIT SOME IDEA OF THE FOREWORD S AUTHOR WHICH HE CAN FIND NO OPPORTUNITY TO PRINT ELSEWHERE. PRAGMATISM ALONE WAS THE STIMULUS OF MY SUGGESTION THAT ELEVEN YEAR OLD DAVID PUTNAM GO ON ONE LEG O

Building the trigram model

In [71]:
#method to build the trigram model
def makeTrigramModel(book):
    #create a default int dictionary
    trigramModel = collections.defaultdict(int)

    for i in range(len(book) - 2):
        #this is getting the 3 char sequence
        trigram = text[i:i+3]
        #increment the count
        trigramModel[trigram] += 1
    #retun the built trigram model
    return trigramModel

In [72]:
trigramModel = makeTrigramModel(cleanedText)
print(trigramModel)

defaultdict(<class 'int'>, {'The': 156, 'he ': 881, 'e P': 11, ' Pr': 8, 'Pro': 5, 'roj': 3, 'oje': 3, 'jec': 7, 'ect': 35, 'ct ': 10, 't G': 3, ' Gu': 5, 'Gut': 3, 'ute': 13, 'ten': 25, 'enb': 4, 'nbe': 5, 'ber': 23, 'erg': 12, 'rg ': 3, 'g e': 6, ' eB': 2, 'eBo': 3, 'Boo': 4, 'ook': 70, 'ok ': 41, 'k o': 20, ' of': 328, 'of ': 298, 'f D': 1, ' Da': 19, 'Dav': 12, 'avi': 21, 'vid': 13, 'id ': 31, 'd g': 23, ' go': 75, 'goe': 11, 'oes': 17, 'es ': 217, 's v': 16, ' vo': 14, 'voy': 5, 'oya': 6, 'yag': 6, 'agi': 4, 'gin': 22, 'ing': 365, 'ng\n': 25, 'g\n ': 1, '\n  ': 73, '   ': 1726, '  \n': 1, ' \nT': 1, '\nTh': 60, 'Thi': 20, 'his': 99, 'is ': 233, 's e': 19, ' eb': 2, 'ebo': 3, 'boo': 34, 'k i': 11, ' is': 172, 's f': 48, ' fo': 154, 'for': 147, 'or ': 241, 'r t': 139, ' th': 1191, 'the': 1181, 'e u': 15, ' us': 29, 'use': 44, 'se ': 77, 'e o': 140, 'f a': 33, ' an': 576, 'any': 63, 'nyo': 2, 'yon': 9, 'one': 137, 'ne ': 133, 'e a': 228, 'nyw': 7, 'ywh': 2, 'whe': 47, 'her': 231, 'er

TASK 2

In [73]:
#get the trigram counts for each matching trigram
def getCounts(model, trigrams):
    counts = []
    for trigram in trigrams:
        counts.append(model[trigram])
    return counts

In [74]:
#Get the matching trigrams that start with the 2 characters
def getMatchingTrigrams(model, twoChars):
    matchingTrigrams = []

    for trigram in model:
        if trigram.startswith(twoChars):
            matchingTrigrams.append(trigram)

    if matchingTrigrams:
        counts = getCounts(model, matchingTrigrams)
        return matchingTrigrams, counts
    else:
        return None, None

In [75]:
#pick the next char of a trigram based on the random counts choice
def pickNextCharacter(trigrams, counts):
    #return only the third character
    chosenTrigram = random.choices(trigrams, counts, k=1)[0]
    return chosenTrigram[-1]

Generate the Text by using the Trigram model created earlier

In [76]:
def generateText(model, startText="TH", length=10000):
    #start the generated text by storing the initial string given in the task
    generatedText = startText
    #keep going till the text is 10000
    while len(generatedText) < length:
        #take out the last 2 chars from the current text
        lastTwoChars = generatedText[-2:]
        #call the method that gets all the trigrams that start with the current 2 chars and their counts
        trigrams, counts = getMatchingTrigrams(model, lastTwoChars)
        #if there are matching trigrams then pick the next char based on the probablity of the counts
        if trigrams:
            nextCharacter = pickNextCharacter(trigrams, counts)
            generatedText += nextCharacter
        #if there are no mathcing trigrams stop generating the text
        else:
            break

    return generatedText

In [77]:
genText = generateText(trigramModel, startText="TH", length=10000)
print(genText)

THE Shem, all ber, thimme friped at
twe wern I sol
ot It I cot makin inut a leacest se a wought, of
cover.
Soce Frid frocke and boar the re was opy, sped in.
At then GRATIONS I mards andry hing offerippet/ fis wingwase seleva shight coome win the we smards
the ing sedgin all smand preept milea knettoon ros, bely shit pire hey ropenot and on the bod shich speoper the
tone. And and
therens blake ronliteem apt in hen wen wer, we bland untle andid her roup ris onet to Onlyin therrigh witent the piecidding antrom gue littany he re sh so sts, liddink, ne I watch. And, on us dy ch, Don ant thes egy lis and
upt frow they mostleselthe seachisee ing
fing hund loateentle mally dis Equitilestione
bels luit put in thersitis any he pird and
bem and ofter in boy in and we seet salf to mal hes pre. Blang whin thead the min Charrater expery octumbecks,
a nig ust seate, th a ropeareep them wersand
th anhor shis one shinharfarld re we somitend mocatee le fromat pre hip any wo kineto he goven dralled win.

Task 3 

In [78]:
#get all the words from words.txt and store them in a set to get easily
def getAllTheWords(filePath):
    with open(filePath, 'r') as file:
        words = set(line.strip().upper() for line in file)
    return words

In [79]:
#take the words out 
def takeOutWords(text):
    #split by spaces
    return text.split()

In [80]:
#count the valid words and calculate the percentage
def calculatePercentage(text, listOfWords):
    words = takeOutWords(text)
    wordCount = 0
    totalWords = len(words)

    for word in words:
        if word in listOfWords:
            wordCount += 1
    
    if totalWords == 0:
        return 0
    
    return (wordCount/totalWords)*100

In [81]:
englishWordList = getAllTheWords('words.txt')
percentage = calculatePercentage(genText, englishWordList)
print(percentage)

0.38105606967882416


## Task 4 Export the Model as JSON

In [ ]:
with open('trigrams.json', 'w') as file:
    json.dump(trigramModel, file)